In [45]:
#INPUT

INPUT = """%gv -> lq, pm
%rv -> jd, nh
%nh -> rs, jd
&vt -> tj
%zv -> pm, gv
%gh -> jd, vd
%hh -> bf, qm
%kx -> nf
%st -> pm, zc
%bh -> qm, pv
&sk -> tj
%hl -> nf, pn
%mt -> st, pm
&jd -> ts, gh, vd, dc, xc
%zm -> hm
%pv -> vv
%zf -> nf, cz
&xc -> tj
%bf -> qm
%ts -> sg
%ht -> ch, nf
%pb -> rv, jd
%nx -> fc
%mb -> mt
%mh -> jd, pb
%lc -> bh
%xg -> mb, pm
%vd -> dc
broadcaster -> gh, dl, xg, fb
%sg -> mh, jd
%qq -> ts, jd
%dl -> nf, sv
%vv -> sm, qm
%zc -> tb
%sr -> zv, pm
%dc -> gb
%cz -> nf, zm
%rs -> jd
%hm -> nf, hl
%gd -> sr
&qm -> lc, pv, nx, fb, kk
&tj -> rx
%gb -> qq, jd
%xf -> zf
%tb -> lg
%sm -> qm, hh
%fb -> dr, qm
%lq -> pm
&nf -> zm, dl, ch, xf, vt
&pm -> sk, zc, tb, gd, mb, xg
%pn -> nf, kx
%fc -> xb, qm
%ch -> xf
&kk -> tj
%lg -> pm, gd
%sv -> nf, ht
%xb -> qm, lc
%dr -> nx, qm"""

In [43]:
# SAMPLE INPUT

INPUT = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

INPUT = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

In [46]:
# Part 1
from collections import defaultdict

rows = INPUT.splitlines()

flip_flop_state = defaultdict(bool)
conj_state = defaultdict(dict)
modules = {}

BROADCAST = 0
FLIP_FLOP = 1
CONJ = 2

inputs = defaultdict(set)

for row in rows:
    module, outputs = row.split(" -> ")
    if module == "broadcaster":
        modules[module] = (BROADCAST, outputs.split(", "))
    elif module[0] == "%":
        modules[module[1:]] = (FLIP_FLOP, outputs.split(", "))
    elif module[0] == "&":
        modules[module[1:]] = (CONJ, outputs.split(", "))

    for output in outputs.split(", "):
        inputs[output].add(module)

for row in rows:
    module, outputs = row.split(" -> ")
    if module[0] == "&":
        for input in inputs[module[1:]]:
            conj_state[module[1:]][input[1:]] = False

low_pulses = 0
high_pulses = 0

print(conj_state)
print(modules)


for i in range(1000):
    queue = [("broadcaster", False, "BUTTON")]

    while queue:
        # print (queue)
        module, state, sender = queue.pop(0)
        # print (sender, "-high->" if state else "-low->", module)

        if state:
            high_pulses += 1
        else:
            low_pulses += 1

        if module not in modules:
            continue

        if modules[module][0] == BROADCAST:
            for output in modules[module][1]:
                queue.append((output, state, module))
        elif modules[module][0] == FLIP_FLOP:
            if state:
                continue
            flip_flop_state[module] = not flip_flop_state[module]
            for output in modules[module][1]:
                queue.append((output, flip_flop_state[module], module))
        elif modules[module][0] == CONJ:
            conj_state[module][sender] = state
            all_high = True
            for last_state in conj_state[module].values():
                if not last_state:
                    all_high = False
                    break
            for output in modules[module][1]:
                queue.append((output, not all_high, module))
        else:
            5/0

print (low_pulses, high_pulses, low_pulses * high_pulses)


defaultdict(<class 'dict'>, {'vt': {'nf': False}, 'sk': {'pm': False}, 'jd': {'gb': False, 'gh': False, 'pb': False, 'rs': False, 'sg': False, 'rv': False, 'nh': False, 'mh': False, 'qq': False}, 'xc': {'jd': False}, 'qm': {'fc': False, 'dr': False, 'sm': False, 'hh': False, 'fb': False, 'bh': False, 'vv': False, 'xb': False, 'bf': False}, 'tj': {'xc': False, 'kk': False, 'sk': False, 'vt': False}, 'nf': {'kx': False, 'zf': False, 'hm': False, 'sv': False, 'ht': False, 'cz': False, 'dl': False, 'pn': False, 'hl': False}, 'pm': {'sr': False, 'xg': False, 'lq': False, 'lg': False, 'st': False, 'zv': False, 'mt': False, 'gv': False}, 'kk': {'qm': False}})
{'gv': (1, ['lq', 'pm']), 'rv': (1, ['jd', 'nh']), 'nh': (1, ['rs', 'jd']), 'vt': (2, ['tj']), 'zv': (1, ['pm', 'gv']), 'gh': (1, ['jd', 'vd']), 'hh': (1, ['bf', 'qm']), 'kx': (1, ['nf']), 'st': (1, ['pm', 'zc']), 'bh': (1, ['qm', 'pv']), 'sk': (2, ['tj']), 'hl': (1, ['nf', 'pn']), 'mt': (1, ['st', 'pm']), 'jd': (2, ['ts', 'gh', 'vd', 'd

In [62]:
# Part 2

from math import lcm

from collections import defaultdict

rows = INPUT.splitlines()

flip_flop_state = defaultdict(bool)
conj_state = defaultdict(dict)
modules = {}

BROADCAST = 0
FLIP_FLOP = 1
CONJ = 2

inputs = defaultdict(set)

for row in rows:
    module, outputs = row.split(" -> ")
    if module == "broadcaster":
        modules[module] = (BROADCAST, outputs.split(", "))
    elif module[0] == "%":
        modules[module[1:]] = (FLIP_FLOP, outputs.split(", "))
    elif module[0] == "&":
        modules[module[1:]] = (CONJ, outputs.split(", "))

    for output in outputs.split(", "):
        inputs[output].add(module)

for row in rows:
    module, outputs = row.split(" -> ")
    if module[0] == "&":
        for input in inputs[module[1:]]:
            conj_state[module[1:]][input[1:]] = False

pre_rx = {}
for input in inputs["rx"]:
    for pre_input in inputs[input[1:]]:
        pre_rx[pre_input[1:]] = -1

i = 0
while True:
    rx_count = [0,0]
    i += 1
    queue = [("broadcaster", False, "BUTTON")]

    while queue:
        # print (queue)
        
        module, state, sender = queue.pop(0)
        if module == "rx":
            if state:
                rx_count[1] += 1
            else:
                rx_count[0] += 1
        # print (sender, "-high->" if state else "-low->", module)

        if state:
            high_pulses += 1
        else:
            low_pulses += 1

        if module not in modules:
            continue

        if modules[module][0] == BROADCAST:
            for output in modules[module][1]:
                queue.append((output, state, module))
        elif modules[module][0] == FLIP_FLOP:
            if state:
                continue
            flip_flop_state[module] = not flip_flop_state[module]
            for output in modules[module][1]:
                queue.append((output, flip_flop_state[module], module))
        elif modules[module][0] == CONJ:
            conj_state[module][sender] = state
            all_high = True
            for last_state in conj_state[module].values():
                if not last_state:
                    all_high = False
                    break
            if not all_high and module in pre_rx and pre_rx[module] == -1:
                pre_rx[module] = i
            for output in modules[module][1]:
                queue.append((output, not all_high, module))
        else:
            5/0
    
    if not i % 100000:
        print (i, pre_rx)

    pre_values = pre_rx.values()
    if -1 not in pre_values:
        print(pre_rx)
        print (lcm(*pre_values))
        break


{'xc': 4057, 'kk': 3931, 'sk': 3917, 'vt': 3943}
246313604784977
